In [1]:
# Imports

import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
import torch

from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd

### Load Dataset

In [2]:
# # Load your dataset
# df = pd.read_csv('../../dataset/reddit_cleansed_data.csv')

# # Calculate weighted score (how good a post is)
# df['weighted_score'] = df['score'] + (10 * df['num_comments']) + (100 * df['gilded_count']) 

# # view all posts where score over 500
# df
# # df[df['weighted_score'] > 1000]

### Load Evaluation Metric (BERTScore)

In [3]:
# Load BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

### Pre-Compute and Save Embeddings for Dataset

In [4]:
def embed_text(text):
    """Generate BERT embeddings for the given text"""
    with torch.no_grad():  # Disable gradient calculations for efficiency
        tokens = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
        outputs = model(**tokens)
        embeddings = outputs.last_hidden_state.mean(dim=1)
        return embeddings.cpu().numpy()  # Move to CPU and convert to numpy

# # Compute embeddings and collect scores
# embeddings = []
# scores = []
# for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Computing embeddings"):
#     story = row['title'] + ' ' + row['selftext']
#     embedding = embed_text(story)
#     embeddings.append(embedding[0])  # embedding is a 1x768 numpy array; we take the first element
#     scores.append(row['weighted_score'])

# # Combine embeddings and scores into a DataFrame
# embeddings_df = pd.DataFrame({
#     'embedding': list(embeddings),
#     'weighted_score': scores
# })

# # Save the DataFrame as a Pickle file
# embeddings_df.to_pickle('story_embeddings_with_scores.pkl')


### Evalulate Story

In [10]:
# Load the DataFrame
embeddings_df = pd.read_pickle('story_embeddings_with_scores.pkl')
embeddings_df.head(2)

,embedding,weighted_score
0,"[0.15318204, 0.017788239, 0.23121382, 0.050800...",115.0
1,"[0.038681276, 0.13021308, 0.1265131, -0.060362...",22.0


In [12]:
# def eval_story(new_story):
#     # Embed the new story
#     new_story_embedding = embed_text(new_story)[0]  # Get the first element of the embedding

#     # Calculate similarities and weighted scores
#     similarities = []
#     for index, row in embeddings_df.iterrows():
#         story_embedding = row['embedding']
#         similarity = cosine_similarity([new_story_embedding], [story_embedding])[0, 0]
#         weighted_score = row['weighted_score']
#         similarities.append((similarity, weighted_score))

#     # Calculate a final score for the new story
#     # This can be a simple average of the weighted scores weighted by similarity
#     if similarities:
#         total_weighted_score = sum(similarity * score for similarity, score in similarities)
#         total_similarity = sum(similarity for similarity, _ in similarities)
#         final_score = total_weighted_score / total_similarity if total_similarity else 0
#     else:
#         final_score = 0

#     return final_score



def eval_story(new_story):
    # Embed the new story
    new_story_embedding = embed_text(new_story)[0]  # Get the first element of the embedding

    # Initialize variables to track the most similar story
    max_similarity = -1  # Start with a similarity that's lower than the lowest possible
    most_similar_score = 0

    # Iterate through each story in the dataset
    for index, row in embeddings_df.iterrows():
        story_embedding = row['embedding']
        similarity = cosine_similarity([new_story_embedding], [story_embedding])[0, 0]
        
        # Check if this story is more similar than the current most similar
        if similarity > max_similarity:
            max_similarity = similarity
            most_similar_score = row['weighted_score']

    return most_similar_score


In [16]:
# Evalulate story

# story = "My cat had a botfly in his eye, so I had to pull it out. One hour and three feet later and I'm still pulling."
# story = "All my life, my parents have told me not to open the basement door, but I got curious and disobeyed them. What is that glowing ball in the sky and why does it hurt my eyes?"
story = "I ate hamburger. But I didn’t realize it was hamburger."
print(eval_story(story))

163.0
